In [ ]:
import requests,time
import os,sys,cv2,random
from lxml import etree
from PIL import Image


headers={'User-Agent':'Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5',
        'Referer':'http://www.51yuansu.com/',}
proxies=['HTTP://163.204.244.198:9999', 'HTTP://163.204.247.33:9999', 'HTTP://58.253.157.138:9999', 'HTTP://163.204.247.249:9999', 'HTTP://112.85.170.187:9999', 'HTTP://163.204.244.228:9999', 'HTTP://60.13.42.39:9999', 'HTTP://112.85.164.30:9999', 'HTTP://163.204.244.226:9999', 'HTTP://163.204.247.21:9999', 'HTTP://171.11.28.163:9999', 'HTTP://163.204.242.170:9999', 'HTTP://111.231.94.44:8888', 'HTTP://112.81.68.50:8118', 'HTTP://121.69.46.177:9000', 'HTTP://222.240.184.126:8086', 'HTTP://218.64.69.79:8080', 'HTTP://218.108.175.15:80', 'HTTP://121.15.254.156:888', 'HTTP://218.66.253.146:8800', 'HTTP://123.117.38.94:9000', 'HTTP://218.95.37.252:3128', 'HTTP://113.65.21.184:9797', 'HTTP://182.88.148.63:9797']


def get_html(url):
    print(random.choice(proxies))
    html=requests.get(url,headers,proxies=random.choice(proxies)).text
    time.sleep(0.5)
    return html


def get_page_url(data):
    '''提取详情页'''
    html=etree.HTML(data)
    url_list=html.xpath('//a[@class="img-wrap"]/@href')
    return url_list


def get_img_url(data):
    '''提取高清大图URL'''
    html=etree.HTML(data)
    url=html.xpath('//img[@class="show-image"]/@src')[0]
    title=html.xpath('//img[@class="show-image"]/@alt')[0]+'.jpg'
    return url,title

def get_img(url,file,path):
    '''下载图片'''
    file_name=path+'\\'+file
    img=requests.get(url,headers=headers).content
    with open(file_name,'wb') as save_img:
        save_img.write(img)
        
def re_backcolor(path,filename,new_path):
    '''图片去除背景色'''
    img_path=path+'\\'+filename
    print(img_path)
    str_list=filename.split('.')
    new_path=new_path+'\\'+str_list[0]+'.png'
    print(new_path)
    img = Image.open(img_path)
    img = img.convert("RGBA")  # 转换获取信息
    pixdata = img.load()
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            if pixdata[x, y][0] > 220 and pixdata[x, y][1] > 220 and pixdata[x, y][2] > 220 and pixdata[x, y][3] > 220:
                pixdata[x, y] = (255, 255, 255, 0)
    img.save(new_path)
    
def read_dir(path):
    if not os.path.exists(path):
        os.mkdir(path)

print("请输入起点网页")
print("注意格式一定要正确：")
print('例如：http://www.51yuansu.com/subject/118-1.html，为了更好的拼凑url，只需要输入‘1.html’之前的即可')
webpage=input()
print("请输入原图保存路径")
origin_path=input()
print("请输入完成图路径")
over_path=input()
read_dir(origin_path)
read_dir(over_path)
print("请输入总页数")
countPage=int(input())
print('是否需要去除背景色，需要请输入y，不需要请输入n')
choose_color=input()
countPage=countPage
i=0
for x in range(countPage):
    x=x+1
    main_url=webpage+str(x)+'.html'
    print('正在爬取第'+str(x)+'页'+'地址为'+main_url)
    html=get_html(main_url)
    url_list=get_page_url(html)
    for url in url_list:
        i=i+1
        if i%150==0:
            print('共爬取了'+str(i)+'张图，服务器需要休息30s')
            time.sleep(30)  #如果爬取了100张图，则休息30s
        html=get_html(url)
        img_url,img_title=get_img_url(html)
        get_img(img_url,img_title,origin_path)
        print('正在下载{}'.format(img_title))
if choose_color == 'y':
    dir_path=origin_path
    dir_path=os.path.join(dir_path) 
    img_list=os.listdir(dir_path)
    for img in img_list:
        re_backcolor(dir_path,img,over_path)
else:
    print('程序结束')